In [2]:
!pip install open_clip_torch transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00


In [3]:
import open_clip
from PIL import Image
import torch

In [5]:
model, _, transform = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k"
)
model.cuda()

CoCa(
  (text): TextTransformer(
    (token_embedding): Embedding(49408, 768)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_final): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (visual): VisionTransformer(
    (patchnorm_pre_ln): Identity()
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(1

In [6]:
def c(new_captions,path_names):
    for path_name in path_names:
      im = Image.open(path_name).convert("RGB")
      im = transform(im).unsqueeze(0).cuda()
      with torch.no_grad(), torch.cuda.amp.autocast():    
        generated = model.generate(im)
      caption = open_clip.decode(generated[0])
      caption = caption.replace("<start_of_text>", "").replace("<end_of_text>", "").strip()
      new_captions.append(caption)
    return new_captions

# GPT2

In [7]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

In [8]:
model1 = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor1 = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer1 = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device1 = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device1)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [9]:
max_length = 16
num_beams = 4
num_beam_groups= 2
# penalty_alpha=0.5
temperature=0.1
def predict_step(new_captions,image_paths,diversity,penalty,temp,topp,topk):
  gen_kwargs = {"max_length": max_length, "num_beams": num_beams, "num_beam_groups":num_beam_groups, "diversity_penalty":diversity,"penalty_alpha":penalty,"temperature":temp,"top_p":topp,"top_k":topk}
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor1(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device1)
  

  output_ids = model1.generate(pixel_values, **gen_kwargs)

  preds = tokenizer1.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  new_captions.append(preds)
  return new_captions

In [11]:
x=int(input("Number of captions to be generated"))
cations=[]
path=["Image1.png"]
if x==1:
  c(cations,path)
elif x>=1:
  c(cations,path)
  y=x-1
  d=0
  p=1
  topp=1
  topk=50
  temp=1
  for i in range(y):
    predict_step(cations,path,d,p,temp,topp,topk)
    d+=0.25
    p-=0.2
    temp-=0.2
    topp+=0.5
    topk+=20
  pass


Number of captions to be generated3


In [12]:
cations

['a soccer player running on a field .',
 ['a soccer player in action on the field'],
 ['a man kicking a soccer ball on a field']]